In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import argparse
from pathlib import Path
import sys
import os

# Add src to path to import feature engineer
sys.path.insert(0, str(Path.cwd().parent.parent / "src"))
from handflow.features.feature_engineer import FeatureEngineer
from handflow.utils.config import load_config

# MediaPipe Hand Connections (Indices)
HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),    # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),    # Index
    (0, 9), (9, 10), (10, 11), (11, 12), # Middle
    (0, 13), (13, 14), (14, 15), (15, 16), # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky
]

def load_from_directory(dir_path):
    """Load raw .npy frames from a directory and return as (T, 84)"""
    path = Path(dir_path)
    files = sorted([f for f in path.glob("*.npy")], key=lambda x: int(x.stem))
    if not files:
        print(f"No .npy files found in {dir_path}")
        return None
        
    frames = []
    for f in files:
        frames.append(np.load(f))
    
    return np.array(frames) # (T, 84)

def visualize_sequence(input_path,split='X_train'):
    """
    Loads data (Raw Dir or Processed NPZ), standardizes it, and animates it.
    """
    path = Path(input_path)
    
    config = load_config('/Users/huynhhuy/Documents/HandFlow/config/config.yaml')
    engineer = FeatureEngineer()
    
    seq_features = None
    
    if path.is_dir():
        print(f"Loading raw sequence from: {path}")
        raw_seq = load_from_directory(path) # (T, 84)
        if raw_seq is None: return
        
        # Apply Normalization Logic (Wrist Centering)
        print("Applying Feature Engineering (Wrist Centering)...")
        seq_features = engineer.transform(raw_seq) # (T, 67)
        
    else:
        print("Invalid input. Must be a directory (raw sequence) or .npz file.")
        return

    # Feature 0-63: XYZ (Relative)
    # Feature 63-66: Distances
    # Feature 67-69: Raw Thumb (4)
    # Feature 70-72: Raw Index MCP (5)
    # Feature 73-75: Raw Index Tip (8)
    
    seq_flat = seq_features
    T = seq_flat.shape[0]
    
    # Reshape features to (T, 21, 3)
    landmarks = seq_flat[:, :63].reshape(T, 21, 3)
    
    # Extract Raw Points
    raw_thumb = seq_flat[:, 67:70]
    raw_index_mcp = seq_flat[:, 70:73]
    raw_index_tip = seq_flat[:, 73:76]
    
    # Setup Plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=-90, azim=-90) # Top-down view approx
    
    # Determine bounds including all global points
    all_points = np.concatenate([
        landmarks.reshape(-1, 3), 
        raw_thumb,
        raw_index_mcp,
        raw_index_tip
    ], axis=0)
    
    min_x, max_x = all_points[:, 0].min(), all_points[:, 0].max()
    min_y, max_y = all_points[:, 1].min(), all_points[:, 1].max()
    min_z, max_z = all_points[:, 2].min(), all_points[:, 2].max()
    
    # Expand bounds slightly
    margin = 0.1
    min_x -= margin; max_x += margin
    min_y -= margin; max_y += margin

    def update(frame):
        ax.clear()
        ax.set_title(f"Frame {frame}/{T}\nRed: Scaled Hand | Global: Thumb(Green), IndexMCP(Cyan), IndexTip(Magenta)")
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        
        # Set consistent fixed limits
        ax.set_xlim(min_x, max_x)
        ax.set_ylim(min_y, max_y)
        ax.set_zlim(min_z, max_z)
        
        # Get Frame Data
        lms = landmarks[frame]
        rt = raw_thumb[frame]
        rim = raw_index_mcp[frame]
        rit = raw_index_tip[frame]
        
        # Plot Joints (Relative Hand)
        ax.scatter(lms[:, 0], lms[:, 1], lms[:, 2], c='red', s=20, alpha=0.6)
        
        # Plot Wrist specifically
        ax.scatter(lms[0, 0], lms[0, 1], lms[0, 2], c='blue', s=50, label='Wrist (Origin)')
        
        # Plot Raw Global Points
        ax.scatter(rt[0], rt[1], rt[2], c='green', s=100, marker='*', label='Raw Thumb')
        ax.scatter(rim[0], rim[1], rim[2], c='cyan', s=80, marker='^', label='Raw Index MCP')
        ax.scatter(rit[0], rit[1], rit[2], c='magenta', s=80, marker='^', label='Raw Index Tip')
        
        # Plot Bones
        for start, end in HAND_CONNECTIONS:
            xs = [lms[start, 0], lms[end, 0]]
            ys = [lms[start, 1], lms[end, 1]]
            zs = [lms[start, 2], lms[end, 2]]
            ax.plot(xs, ys, zs, c='black', alpha=0.5)
            
        ax.legend()

    anim = FuncAnimation(fig, update, frames=T, interval=100)
    plt.show()


In [7]:
from IPython.display import HTML, display
import matplotlib.animation as animation

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import argparse
from pathlib import Path
import sys
import os

# Add src to path to import feature engineer
sys.path.insert(0, str(Path.cwd().parent.parent / "src"))
from handflow.features.feature_engineer import FeatureEngineer
from handflow.utils.config import load_config

# MediaPipe Hand Connections (Indices)
HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),    # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),    # Index
    (0, 9), (9, 10), (10, 11), (11, 12), # Middle
    (0, 13), (13, 14), (14, 15), (15, 16), # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky
]

def load_from_directory(dir_path):
    """Load raw .npy frames from a directory and return as (T, 84)"""
    path = Path(dir_path)
    files = sorted([f for f in path.glob("*.npy")], key=lambda x: int(x.stem))
    if not files:
        print(f"No .npy files found in {dir_path}")
        return None
        
    frames = []
    for f in files:
        frames.append(np.load(f))
    
    return np.array(frames) # (T, 84)

def plot_hand_2d(landmarks, ax, title='Hand Landmarks'):
    """
    Plot 2D hand landmarks with connections.
    
    Args:
        landmarks: np.array of shape (21, 4) - x, y, z, visibility
        ax: matplotlib axis
        title: plot title
    """
    # Extract x, y coordinates
    x = landmarks[:, 0]
    y = landmarks[:, 1]
    
    # Plot connections
    for connection in HAND_CONNECTIONS:
        start, end = connection
        ax.plot([x[start], x[end]], [y[start], y[end]], 'b-', linewidth=1.5, alpha=0.6)
    
    # Plot landmarks with color coding by finger
    colors = ['red'] + ['orange']*4 + ['green']*4 + ['blue']*4 + ['purple']*4 + ['pink']*4
    ax.scatter(x, y, c=colors, s=50, zorder=5)
    
    # Annotate wrist
    ax.annotate('Wrist', (x[0], y[0]), textcoords='offset points', xytext=(5, 5), fontsize=8)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(title)
    ax.invert_yaxis()  # Match image coordinates
    ax.set_aspect('equal')

def create_gesture_animation(path, title='Gesture Animation'):
    """
    Create an animation of a gesture sequence.
    
    Args:
        sequence: np.array of shape (12, 84)
    
    Returns:
        matplotlib animation object
    """
    sequence = load_from_directory(path)
     
    fig, ax = plt.subplots(figsize=(8, 8))
    
    def animate(frame_idx):
        ax.clear()
        frame = sequence[frame_idx]
        landmarks = frame.reshape(21, 4)
        plot_hand_2d(landmarks, ax, title=f"{title} - Frame {frame_idx + 1}/12")
        return ax,
    
    anim = animation.FuncAnimation(fig, animate, frames=12, interval=100, blit=False)
    plt.close()
    return anim

In [8]:
def graph_feature(path):
    raw_seq = load_from_directory(path)
    
    engineer = FeatureEngineer()
    seq_features = engineer.transform(raw_seq)

    return seq_features
    

In [10]:
# Create animation for a gesture

anim = create_gesture_animation("/Users/huynhhuy/Documents/HandKey/data/raw/right_mp_data/touch/100", title="test")
display(HTML(anim.to_jshtml()))

features = graph_feature("/Users/huynhhuy/Documents/HandKey/data/raw/right_mp_data/touch/100")